In [1]:
# example code
# uses pre-aligned files 

In [13]:
#python imports - python version is 3.7.3
import os
from collections import Counter 
import numpy as np #1.17.0
import pandas as pd #0.25.0
import pydicom #1.3.0
from matplotlib import pyplot as plt #matplotlib 3.1.1
from PIL import Image #Pillow 6.1.0
from functools import reduce
import cv2 #opencv-python 4.1.0.25
import sklearn #0.2.1
from sklearn.preprocessing import MinMaxScaler
import imageio #2.5.0
import scipy #1.3.0
from fastai.basic_train import *
from fastai.basic_train import *

ModuleNotFoundError: No module named 'fastai'

In [ ]:
conda install jupyter notebook

In [3]:
#local imports 
from parsing_VOI import *

In [4]:
#first create new file called 'active_jpegs' and
# create jpegs for all these files

In [5]:
#input path to files
patient_path=r'C:\Users\sanfordt\Desktop\jupyter_notebook\demo'

In [6]:
def order_dicom(dicom_file_list):
    '''
    function to sort dicom files in z space. Relys on pydicom ds.SliceLocation
    :param dicom_file_list --> full path to dicom files to sort
    :return list of file paths sorted from apex to base of prostate in axial plane
    '''
    dicoms={}
    for path in dicom_file_list:
        file=path
        ds=pydicom.read_file(path)
        dicoms[str(file)] = float(ds.SliceLocation)
    updated_imagelist=[key for (key, value) in sorted(dicoms.items(), key=lambda x: x[1])]
    return(updated_imagelist)

In [7]:
def segment_image(path_to_image,vals,patient_dir,index,series,pad=10):
    '''
    helper function that takes in path to image, values and performs the segmentation
    :param path_to_image: full path to dicom file
    :param vals: indexes
    :param patient_dir:
    :param index:
    :param series:
    :param pad: number of voxes around the image in question
    :return:
    '''
    ds = pydicom.dcmread(path_to_image)
    data = ds.pixel_array
    data=rescale_array(data)  #normalize by slice

    print('The image has {} x {} voxels'.format(data.shape[0],data.shape[1]))
    data_downsampled = data[vals[2] - pad:vals[4] + pad, vals[1] - pad:vals[3] + pad]
    print('The cropped image has {} x {} voxels'.format(
        data_downsampled.shape[0], data_downsampled.shape[1]))

    ds.PixelData = data_downsampled.tobytes()
    ds.Rows, ds.Columns = data_downsampled.shape

    if not os.path.exists(os.path.join(patient_path,'T2', patient_dir)):
        os.mkdir(os.path.join(patient_path,'T2', patient_dir))
    os.chdir(os.path.join(patient_path,'T2',patient_dir))

    #save image
    #if series == 't2':
    #    ds.save_as(patient_dir + '_' + str(index) + '_T2_' + vals[0] + '.dcm')

    return data_downsampled

In [8]:
def voi_paths(filetype='PIRADS'):
    '''
    function to find all .voi files within a specific patient directory. 
    :param filetype (str): type of segementation to perform
           PIRADS - tumor
           wp- whole prostate
           u -urethra
    :return: list of files that have the search term of interest (i.e. 'PIRADS')
    '''
    voi_dir=os.path.join(patient_path,'data', 'voi')
    file_path=None
    for file in os.listdir(voi_dir):
        split_file=file.replace(' ','_').split('_')
        if filetype not in split_file:
            continue
        elif filetype in split_file:
            file_path=str(os.path.join(voi_dir,file))
    return file_path

In [9]:
def dicom_paths():
    '''
    start in patient directory, then find all dicom files listed in that directory -->  note adc and highb
    are aligned to t2
    :param patient_dir (str): the folder name of the files
    :return: list of paths to dicom files within patient directory
    '''

    #set base directory
    pat_dir=os.path.join(patient_path,'data','dicoms')

    #set path to each directory for aligned files
    t2_dir=os.path.join(pat_dir,'t2')
    adc_dir=os.path.join(pat_dir,'adc','aligned')
    highb_dir=os.path.join(pat_dir,'highb','aligned')

    #setup dict names
    series_dict={'t2':t2_dir,'adc':adc_dir,'highb':highb_dir}

    #loop over series and joint with series dir path to get paths to dicom files for all images in all series
    dicom_dict={}
    for series in series_dict.keys():
        files=os.listdir(series_dict[series])
        path_list=[]
        for file in files:
            path_list+=[os.path.join(series_dict[series],file)]
        path_list=order_dicom(path_list)
        dicom_dict[series]=path_list
    return dicom_dict

In [10]:
def rescale_array(array):
    scaler=MinMaxScaler(feature_range=(0,255))
    scaler=scaler.fit(array)
    X_scaled=scaler.transform(array)
    return (X_scaled)

In [11]:
def segment_aligned():
    '''
    for files that are already aligned, this looks up dicom paths, voi paths, and segments all the images,
    then saves the T2 and the combination of 3 separately
    :param patient_dir --> output of completed_files
    :return: all segmented files
    '''
    voi_p_paths=voi_paths()
    for voi_path in [voi_p_paths]:
        
        #get name for later use
        name=os.path.basename(os.path.normpath(voi_path))
        dicom_path_list=dicom_paths()
        bboxes = ParseVOI().BBox_from_position(voi_path) #from parsingVOI module

        #start by iterating over bounding boxes
        index = 0
        for bbox in bboxes.keys():
            vals=bboxes[bbox] #select values for each bounding box

            #for each bounding box, select the appropriate slice and segment
            segmented_image_dict={}
            for series in dicom_path_list:
                paths=dicom_path_list[series]
                path_to_load=paths[int(bbox)]
                segmented_image=segment_image(path_to_load,vals,patient_path,index,series)
                segmented_image_dict[series]=segmented_image

            #extract each sequance array and combine into numpy array
            t2=segmented_image_dict['t2']; adc=segmented_image_dict['adc']; highb=segmented_image_dict['highb']
            stacked_image=np.dstack((t2,adc,highb))

            #normalize 
            stacked_image[:,:,0]=rescale_array(stacked_image[:,:,0])
            stacked_image[:, :, 1] = rescale_array(stacked_image[:, :, 1])
            stacked_image[:, :, 2] = rescale_array(stacked_image[:, :, 2])

            #make a directory if one doesn't already exist for images, conver to Image and save .jpg
            if not os.path.exists(os.path.join(patient_path,'data','jpg',name)):
                os.mkdir(os.path.join(patient_path,'data','jpg',name))
            os.chdir(os.path.join(patient_path,'data','jpg',name))

            #opencv solution
            cv2.imwrite(os.path.join(patient_path,'data','jpg',name, str(name) +'_'+str(index) + '_' + vals[0] + '.jpg'),stacked_image)

            index+=1

In [12]:
segment_aligned()

The image has 384 x 384 voxels
The cropped image has 48 x 37 voxels
The image has 384 x 384 voxels
The cropped image has 48 x 37 voxels
The image has 384 x 384 voxels
The cropped image has 48 x 37 voxels
The image has 384 x 384 voxels
The cropped image has 46 x 33 voxels
The image has 384 x 384 voxels
The cropped image has 46 x 33 voxels
The image has 384 x 384 voxels
The cropped image has 46 x 33 voxels
The image has 384 x 384 voxels
The cropped image has 42 x 40 voxels
The image has 384 x 384 voxels
The cropped image has 42 x 40 voxels
The image has 384 x 384 voxels
The cropped image has 42 x 40 voxels
The image has 384 x 384 voxels
The cropped image has 46 x 38 voxels
The image has 384 x 384 voxels
The cropped image has 46 x 38 voxels
The image has 384 x 384 voxels
The cropped image has 46 x 38 voxels


In [26]:
## Deploy model

In [30]:
def apply_model():        
    #import model
    model_path = os.path.join(patient_path,'model')
    learn = load_learner(model_path)
    
    for tumor in os.listdir(os.path.join(test_path)):
        print(tumor)
        sum_pred = np.zeros(4)
        square_pred = np.zeros(4)
        img_num = 0

        for image in sorted(os.listdir(os.path.join(test_path, tumor))):
            img = open_image(os.path.join(test_path, tumor, image))
            pred_class, pred_idx, outputs = learn.predict(img)
            print(outputs.numpy())
            sum_pred += outputs.numpy()
            square_pred += (outputs.numpy()) ** 2
            img_num += 1
        
        # metrics
        average = sum_pred / img_num
        sum_pred_class = np.argmax(sum_pred)
        ave_pred_class = np.argmax(average)
        square_pred_class = np.argmax(square_pred)

        print('sum prediction {}'.format(sum_pred))
        print('average prediction {}'.format(average))
        print('square prediction {}'.format(square_pred))

In [31]:
apply_model()

NameError: name 'load_learner' is not defined